In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
!python --version

Python 3.7.12


In [3]:
! wget https://bj.bcebos.com/paddlex/datasets/optic_disc_seg.tar.gz
! tar xzf optic_disc_seg.tar.gz

--2021-12-04 02:42:13--  https://bj.bcebos.com/paddlex/datasets/optic_disc_seg.tar.gz
Resolving bj.bcebos.com (bj.bcebos.com)... 103.235.46.61, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to bj.bcebos.com (bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19077558 (18M) [application/octet-stream]
Saving to: ‘optic_disc_seg.tar.gz’

optic_disc_seg.tar. 100%[===================>]  18.19M  3.49MB/s    in 15s     

2021-12-04 02:42:30 (1.20 MB/s) - ‘optic_disc_seg.tar.gz’ saved [19077558/19077558]



In [4]:
!python -m pip install paddlepaddle-gpu==2.2.0.post111 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html

Looking in links: https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
     |████████████████████████████████| 409.1 MB 26 kB/s 


In [14]:
import paddle
paddle.utils.run_check()

Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 GPU.
PaddlePaddle works well on 1 GPUs.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


In [5]:
!python -m pip install paddlex==2.0.0 -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 944 kB 271 kB/s 
     |████████████████████████████████| 2.7 MB 8.1 MB/s 
     |████████████████████████████████| 288 kB 10.5 MB/s 
     |████████████████████████████████| 1.5 MB 18.9 MB/s 
     |████████████████████████████████| 6.8 MB 18.5 MB/s 
     |████████████████████████████████| 151 kB 24.2 MB/s 
     |████████████████████████████████| 64 kB 2.3 MB/s 
     |████████████████████████████████| 2.1 MB 15.5 MB/s 
     |████████████████████████████████| 191 kB 30.5 MB/s 
     |████████████████████████████████| 127 kB 44.4 MB/s 
     |████████████████████████████████| 69 kB 4.9 MB/s 
     |████████████████████████████████| 42 kB 832 kB/s 
     |████████████████████████████████| 1.9 MB 24.5 MB/s 
     |████████████████████████████████| 50 kB 4.5 MB/s 
     |████████████████████████████████| 596 kB 17.0 MB/s 
     |████████████████████████████████| 5.6 MB 19.5 MB/s 
     |███████████████████

In [7]:
!python -m pip install cython pycocotools

In [15]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import paddlex as pdx
from paddlex import transforms as T
train_transforms = T.Compose([
    T.Resize(target_size=512),
    T.Normalize()
])
eval_transforms = T.Compose([
    T.Resize(target_size=512),
    T.Normalize()
])

In [16]:
train_dataset = pdx.datasets.SegDataset(
    data_dir='optic_disc_seg',
    file_list='optic_disc_seg/train_list.txt',
    label_list='optic_disc_seg/labels.txt',
    transforms=train_transforms,
    shuffle=True)
eval_dataset = pdx.datasets.SegDataset(
    data_dir='optic_disc_seg',
    file_list='optic_disc_seg/val_list.txt',
    label_list='optic_disc_seg/labels.txt',
    transforms=eval_transforms)

2021-12-04 02:51:22 [INFO]	267 samples in file optic_disc_seg/train_list.txt
2021-12-04 02:51:22 [INFO]	76 samples in file optic_disc_seg/val_list.txt


In [17]:
num_classes = len(train_dataset.labels)
model = pdx.seg.DeepLabV3P(num_classes=num_classes)

model.train(
    num_epochs=40,
    train_dataset=train_dataset,
    train_batch_size=1,
    eval_dataset=eval_dataset,
    save_interval_epochs=1,
    log_interval_steps=2,
    early_stop=True,
    early_stop_patience=5,
    save_dir='output/deeplab')

2021-12-04 02:51:25 [INFO]	Loading pretrained model from output/deeplab/pretrain/model.pdparams
2021-12-04 02:51:26 [WARNING]	[SKIP] Shape of pretrained params head.decoder.conv.weight doesn't match.(Pretrained: [19, 256, 1, 1], Actual: [2, 256, 1, 1])
2021-12-04 02:51:26 [WARNING]	[SKIP] Shape of pretrained params head.decoder.conv.bias doesn't match.(Pretrained: [19], Actual: [2])
2021-12-04 02:51:26 [INFO]	There are 358/360 variables loaded into DeepLabV3P.
2021-12-04 02:51:28 [INFO]	[TRAIN] Epoch=1/40, Step=2/267, loss=0.000000, lr=0.009999, time_each_step=0.63s, eta=1:53:7
2021-12-04 02:51:29 [INFO]	[TRAIN] Epoch=1/40, Step=4/267, loss=0.000000, lr=0.009997, time_each_step=0.51s, eta=1:31:4


RuntimeError: ignored